In [1]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install statsmodels
!{sys.executable} -m pip install googlemaps

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
#import statsmodels.api as sm

In [9]:
def to_hhmmss(seconds):
    hours = seconds / 3600
    seconds -= hours * 3600
    minutes = seconds / 60
    seconds -= minutes * 60
    if hours < 10:
        hh = '0' + str(hours)
    else:
        hh = str(hours % 24)
    if minutes < 10:
        mm = '0' + str(minutes)
    else:
        mm = str(minutes)
    if seconds < 10:
        ss = '0' + str(seconds)
    else:
        ss = str(seconds)
    return hh +':' + mm + ':' + ss

def get_dep_timestamp(f):
    return f['FL_DATE'] + ' ' + to_hhmmss(f['DEP_TIME'])
    
def get_arr_timestamp(f):
    return f['FL_DATE'] + ' ' + to_hhmmss(f['ARR_TIME'])

# Getting our data

### Flights
1. Bureau of Transportation Statistics
2. left-joining lookup tables of flight -> lookup tables for airports -> lookup tables for city markets
3. locations of airport from "city markets" with Google Geocoding -> able to adjust timezones with Google Timezone


### UFOs
1. Kaggle UFO data

In [16]:
flights = pd.read_csv('./data/flights/all_flights.csv')
flights = flights.drop(columns=['FL_NUM', 'ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID'])
flights = flights.loc[:, ~flights.columns.str.contains('^Unnamed')]

ufos = pd.read_csv("data/ufo-sightings/ufos-2013.csv", sep=',', low_memory=False)
ufos['lat'] = ufos['latitude']
ufos['lng'] = ufos['longitude']
ufos = ufos[[ 'datetime', 'duration (seconds)', 'lat', 'lng' ]]
ufos = ufos[ ufos['datetime'] > '2013-01-01 00:00:00' ]

flights['dep_timestamp'] = flights.apply(get_dep_timestamp, axis=1)
flights['arr_timestamp'] = flights.apply(get_arr_timestamp, axis=1)

95220

In [ ]:
results = pd.DataFrame(columns=['ufo_datetime', 'flight_datetime', 'from_origin'])

for _, u in ufos.iterrows():
    for _, f in flights.iterrows():
        d_to_origin = (( (f['o_lat'] - u['lat'])**2 + (f['o_lng'] - u['lng'])**2 ) ** 0.5) * 55.2428
        d_to_dest = (( (f['d_lat'] - u['lat'])**2 + (f['d_lng'] - u['lng'])**2 ) ** 0.5) * 55.2428
        if d_to_origin < 40 and abs(pd.Timestamp(f['dep_timestamp']) - pd.Timestamp(u['datetime'])) < pd.Timedelta(Hour(1)):
            print 'origin ' + u['datetime']
            results.loc[results.size] = [u['datetime'], f['dep_timestamp'], True]
            break
        elif d_to_dest < 40 and abs(pd.Timestamp(f['arr_timestamp']) - pd.Timestamp(u['datetime'])) < pd.Timedelta(Hour(1)):
            print 'dest ' + u['datetime']
            results.loc[results.size] = [u['datetime'], f['arr_timestamp'], True]
            break
    

dest 2013-01-01 00:05:00
origin 2013-01-02 06:20:00
origin 2013-01-03 04:10:00
dest 2013-01-04 18:00:00
dest 2013-01-04 19:00:00
origin 2013-01-05 21:04:00
dest 2013-01-14 21:39:00
dest 2013-01-15 02:20:00
dest 2013-01-17 20:15:00
origin 2013-01-24 16:10:00
origin 2013-01-27 22:57:00
dest 2013-01-31 23:30:00
origin 2013-03-15 20:34:00
dest 2013-03-16 23:00:00
origin 2013-03-20 21:30:00
dest 2013-03-21 18:00:00
dest 2013-03-21 22:15:00
dest 2013-03-22 19:56:00
origin 2013-03-22 22:00:00
dest 2013-03-25 22:00:00
origin 2013-03-28 11:25:00
origin 2013-03-28 23:00:00
dest 2013-03-29 23:30:00
dest 2013-03-29 23:30:00
dest 2013-03-30 22:39:00
origin 2013-04-30 23:10:00
dest 2013-05-01 14:00:00
origin 2013-05-01 21:25:00
origin 2013-05-04 21:30:00
dest 2013-05-04 22:30:00
origin 2013-05-10 22:27:00
dest 2013-05-11 23:58:00
origin 2013-05-14 04:00:00
dest 2013-05-15 00:00:00
dest 2013-05-15 22:00:00
dest 2013-05-17 19:00:00
dest 2013-05-19 22:52:00
origin 2013-05-20 22:30:00
origin 2013-05-20 